Permision  = Calendars.Read
# manager client
https://apps.dev.microsoft.com/#/application/42e2dba6-d417-48d3-8d36-1b4af3ae6e56

# graph api ref
https://developer.microsoft.com/en-us/graph/docs/api-reference
    
# reference 
https://github.com/OfficeDev/outlook-dev-docs/blob/master/docs/rest/python-tutorial.md
    
# get code 
https://login.microsoftonline.com/common/oauth2/v2.0/authorize?scope=openid+Calendars.Read&response_type=code&client_id=42e2dba6-d417-48d3-8d36-1b4af3ae6e56&redirect_uri=http%3A%2F%2Flocalhost%3A8000%2F

In [2]:
client_id = 'pyapi'
client_secret = 'ycnKZI215?jghrJSPL12_%%'

In [3]:
import requests

In [4]:
authority = 'https://login.microsoftonline.com'

# The authorize URL that initiates the OAuth2 client credential flow for admin consent
authorize_url = '{0}{1}'.format(authority, '/common/oauth2/v2.0/authorize?{0}')

# The token issuing endpoint
token_url = '{0}{1}'.format(authority, '/common/oauth2/v2.0/token')

scopes = [ 'openid',
           'Calendars.Read' ]

In [5]:
def get_token_from_code(auth_code, redirect_uri):
  # Build the post form for the token request
  post_data = { 'grant_type': 'authorization_code',
                'code': auth_code,
                'scope': ' '.join(str(i) for i in scopes),
                'client_id': client_id,
                'client_secret': client_secret
              }
              
  r = requests.post(token_url, data = post_data)
  
  try:
    return r.json()
  except:
    return 'Error retrieving token: {0} - {1}'.format(r.status_code, r.text)

In [7]:
code = 'OAQABAAIAAADX8GCi6Js6SK82TsD2Pb7rW_7AgEcFMkyhp8V4EzgwAtbGgMSS7GfA9PkAed9BGvhqVB_5grE0GCQj-moMJ4b7jtgSvQDeKwYQtvdqWvt0K5-ap8Udx8IHKCwdthNHtB8GrfETK-GpUsAQFN2eINU2wLK1KGrCHjdooi-b7RkZoRwcXaTjCtDEmOihTFgxi4OBjP3Nut5OYOXPRystOFQQTlszSJr0b3kJMYOD_h5Oon0V867b5sAWbiVvIPf9rIndT9nrNSiwDMl07b_JB0t2QPnjYxT3WMAROq0jdMKorKNYF4g-WJaXRL2eQzZYhV38mI8Tvtu01ZiVbhdOq_0hS8cLp-UmHpCpK2fWgBMm3OcluMhUAOxKHLWtOL0yUqibMydKoav19WvmbVsLv6XG5KvT33XlZPudwwQFOtmdzCXE4YMEpDSqOWVsSevpx-HjCY-zLB5BcFVAnyiBGuaRzvvP1xEo-3c1rc9k201IcTWX4KK0DzN9tnNFDNUCc6wqUSOGc5wL4xWVAwcFvh5isyEaUq1aA5Mg0LxPl6CrQbkSM-fuPRQe6qLG8gnspSdrFjL1yWcgsgVW5HukQU7EeQr35bNwF-yBcTwupDPNUGaCn8nc75c2BDeZW8b-kDIx5XBBAKX0bdS7CCp6zFWrqeanUojiddOcEMb5LT7k0PEPwatfxDd1R5Zy293UWFkgAA'
get_token_from_code(code, '')

{'correlation_id': '6964846e-288b-4d74-89ea-d03c759d003f',
 'error': 'invalid_grant',
 'error_codes': [70002, 70008],
 'error_description': 'AADSTS70002: Error validating credentials. AADSTS70008: The provided authorization code or refresh token is expired. Send a new interactive authorization request for this user and resource.\r\nTrace ID: 0ba22652-7e67-45a6-a5ab-a19741361000\r\nCorrelation ID: 6964846e-288b-4d74-89ea-d03c759d003f\r\nTimestamp: 2018-05-19 16:37:25Z',
 'timestamp': '2018-05-19 16:37:25Z',
 'trace_id': '0ba22652-7e67-45a6-a5ab-a19741361000'}

In [23]:
import requests
import uuid
import json

graph_endpoint = 'https://graph.microsoft.com/v1.0{0}'

# Generic API Sending
def make_api_call(method, url, token, payload = None, parameters = None):
  # Send these headers with all API calls
  headers = { 'User-Agent' : 'python_tutorial/1.0',
              'Authorization' : 'Bearer {0}'.format(token),
              'Accept' : 'application/json' }
              
  # Use these headers to instrument calls. Makes it easier
  # to correlate requests and responses in case of problems
  # and is a recommended best practice.
  request_id = str(uuid.uuid4())
  instrumentation = { 'client-request-id' : request_id,
                      'return-client-request-id' : 'true' }
                      
  headers.update(instrumentation)
  
  response = None
  
  if (method.upper() == 'GET'):
      response = requests.get(url, headers = headers, params = parameters)
  elif (method.upper() == 'DELETE'):
      response = requests.delete(url, headers = headers, params = parameters)
  elif (method.upper() == 'PATCH'):
      headers.update({ 'Content-Type' : 'application/json' })
      response = requests.patch(url, headers = headers, data = json.dumps(payload), params = parameters)
  elif (method.upper() == 'POST'):
      headers.update({ 'Content-Type' : 'application/json' })
      response = requests.post(url, headers = headers, data = json.dumps(payload), params = parameters)
      
  return response

In [33]:
def get_cal(access_token):
  get_me_url = graph_endpoint.format('/me/calendar/events')

  # Use OData query parameters to control the results
  #  - Only return the displayName and mail fields
  # query_parameters = {'$select': 'displayName,mail'}
  query_parameters = {}

  r = make_api_call('GET', get_me_url, access_token, "", parameters = query_parameters)

  if (r.status_code == requests.codes.ok):
    return r.json()
  else:
    return "{0}: {1}".format(r.status_code, r.text)

events = get_cal(access_token)
    
events['value'][0]['subject']

value
@odata.nextLink
@odata.context


'Brown-Bag Session: Automated Geometry Creation of Airport Runways with Machine Learning'